In [39]:
import os
if "rrszynka" not in os.getcwd():
   raise Exception("Working locally - is this desired?")
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import datetime
import matplotlib.pyplot as plt
import sys
sys.path.append(os.getcwd()+'/../Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()
from bee_helpers import calc_trip_lengths, get_forager_bee_ids, get_random_bee_ids, get_all_bee_ids, get_alive_bees_for_day
import bb_utils
import random
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from ipywidgets import interact

In [40]:
# Setup parameters 
datetime_start = datetime.datetime(2016,7,20)
num_hours = 24
num_intervals_per_hour = 120
bee_ids = get_alive_bees_for_day(datetime_start).bee_id.tolist()

In [41]:
# Get alive bees for the chosen day and filter other (dead) bees out of the presence table
bees_alive_for_day = get_alive_bees_for_day(datetime_start)['bee_id'].values
presence = c.load('PRESENCE-counts-2016-07-20_00_num_hours_24_int_size_120_conf_099_cams_0123', type=CacheType.presence, format=CacheFormat.csv)
presence.index.name = 'bee_id'
# Safety check
presence.shape[0] == len(bees_alive_for_day)

True

In [42]:
bee = 614 #random.choice(bees_alive_for_day)
xs = np.arange(0,presence.shape[1])
ys = presence.loc[bee]

###  Optional filtering of values >90. This step changes everything above 90 to 90. 

In [43]:
ys[ys>90] = 90

### Version 1: Using rolling median only, with variable window size

In [37]:
fig = go.FigureWidget()
scatt = fig.add_scatter(
    mode='markers',
    marker=dict(
        size = 3
    )
)
py.iplot(fig, filename='janek')

In [38]:
@interact(a=(1, 500))
def update(a=1):
    with fig.batch_update():
        ys_rolled = ys.rolling(a).median()
        scatt.x=xs
        scatt.y=ys_rolled

interactive(children=(IntSlider(value=1, description='a', max=500, min=1), Output()), _dom_classes=('widget-in…

### Version 2: Using threshold value only; 

In [22]:
fig = go.FigureWidget()
scatt = fig.add_scatter(
    mode='markers',
    marker=dict(
        size = 3
    )
)
fig

FigureWidget({
    'data': [{'marker': {'size': 3}, 'mode': 'markers', 'type': 'scatter', 'uid': '5ff0e897-88a…

In [28]:
@interact(threshold=(0, 90))
def update(threshold=1):
    with fig.batch_update():
        pts = ys.copy()
        pts[pts>threshold] = 90
        pts[pts<=threshold] = 0
        scatt.x=xs
        scatt.y=pts

interactive(children=(IntSlider(value=1, description='threshold', max=90), Output()), _dom_classes=('widget-in…

In [24]:
ys = presence.loc[bee]

In [25]:
threshold = 20
ys_filtered = ys
ys_filtered[ys>threshold] = 90
ys_filtered[ys<=threshold] = 0
scatt.x=xs
scatt.y=ys_filtered

In [26]:
def score_to_0_or_90(presence_for_bee, threshold=50):
    
    return df

In [27]:
# score_to_0_or_90(ys, threshold = 0)
xxx = score_to_0_or_90(ys)

NameError: name 'df' is not defined